In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS']="--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell"
os.environ["JAVA_HOME"]="/usr/lib/jvm/java-8-openjdk-amd64/"

In [2]:
import sys

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [3]:
sc = SparkContext(appName="PythonSparkStreamingKafka")
sc.setLogLevel("WARN")

In [4]:
ssc = StreamingContext(sc, 60)

In [5]:
#brokers = "localhost:9092"
topic = "twitter"

In [6]:
# kvs = KafkaUtils.createDirectStream(ssc, [topic], {"bootstrap.servers": 'localhost:9092'})
# lines = kvs.map(lambda x: x[1])
# counts = lines.flatMap(lambda line: line.split(" ")) \
#         .map(lambda word: (word, 1)) \
#         .reduceByKey(lambda a, b: a+b)
# counts.pprint()

# ssc.start()
# ssc.awaitTermination()

In [7]:
# import json
# kvs = KafkaUtils.createStream(ssc, [topic], {"bootstrap.servers": 'localhost:9092'})
# lines = kvs.map(lambda x: json.loads(x[1]))

kvs = KafkaUtils.createStream(ssc, 'localhost:9092', 'spark-streaming', {'twitter':1})
lines = kvs.map(lambda x: json.loads(x[1]))

In [8]:
lines.count().map(lambda x:"Tweets in this batch: %s" % x).pprint()

In [9]:
authors_dstream = lines.map(lambda tweet: tweet['user']['screen_name'])

In [10]:
author_counts = authors_dstream.countByValue()
author_counts.pprint()

In [11]:
author_counts_sorted_dstream = author_counts.transform(\
  (lambda foo:foo\
   .sortBy(lambda x:( -x[1]))))

In [12]:
author_counts_sorted_dstream.pprint()

In [13]:
top_five_authors = author_counts_sorted_dstream.transform\
  (lambda rdd:sc.parallelize(rdd.take(5)))
top_five_authors.pprint()

In [14]:
filtered_authors = author_counts.filter(lambda x:\
                                                x[1]>1 \
                                                or \
                                                x[0].lower().startswith('rm'))

In [15]:
filtered_authors.transform\
  (lambda rdd:rdd\
  .sortBy(lambda x:-x[1]))\
  .pprint()

In [16]:
lines.\
    flatMap(lambda tweet:tweet['text'].split(" "))\
    .countByValue()\
    .transform\
      (lambda rdd:rdd.sortBy(lambda x:-x[1]))\
    .pprint()

In [18]:
ssc.start()
ssc.awaitTermination(timeout=180)

-------------------------------------------
Time: 2020-08-05 14:20:00
-------------------------------------------

-------------------------------------------
Time: 2020-08-05 14:20:00
-------------------------------------------

-------------------------------------------
Time: 2020-08-05 14:20:00
-------------------------------------------

-------------------------------------------
Time: 2020-08-05 14:20:00
-------------------------------------------

-------------------------------------------
Time: 2020-08-05 14:20:00
-------------------------------------------

-------------------------------------------
Time: 2020-08-05 14:20:00
-------------------------------------------



KeyboardInterrupt: 